## **Let's construct LeNet in Keras!**

![](https://www.researchgate.net/profile/Sheraz_Khan8/publication/321586653/figure/fig4/AS:568546847014912@1512563539828/The-LeNet-5-Architecture-a-convolutional-neural-network.png)
## **LeNet Architecture**
S.No | Layers | Output Shape (Height, Width, Channels)
--- | --- | ---
1 | Input Layer | 32 x 32 x 1
2 | Conv2d [6 Filters of size = 5x5, stride = 1, padding = 0 ] | 28 x 28 x 6
3 | Average Pooling [stride = 2, padding = 0] | 14 x 14 x 6
4 | Conv2d [16 Filters of size = 5x5, stride = 1, padding = 0 ] | 10 x 10 x 16
5 | Average Pooling [stride = 2, padding = 0] | 5 x 5 x 16
6 | Conv2d [120 Filters of size = 5x5, stride = 1, padding = 0 ] | 1 x 1 x 120
7 | Linear1 Layer | 120 
8 | Linear2 Layer | 84 
9 | Final Linear Layer | 10


### **Loading and preprocessing our Data**

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adadelta
import numpy as np

In [2]:
# load dataset
(x_train,y_train), (x_test, y_test) = mnist.load_data()

img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
print(np.unique(y_test))

input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
#print(np.unique(y_test))
num_classes = y_test.shape[1]
num_pixels = img_rows*img_cols



[0 1 2 3 4 5 6 7 8 9]


In [36]:
"""model = Sequential()

model.add(Conv2D(6, (5,5),
                 padding = 'same',
                 input_shape = input_shape))
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))

model.add(Conv2D(16, (5,5),
                 padding = 'same'))
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))

model.add(Conv2D(120, (5,5),
                 padding = 'same'))
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))

model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])

print(model.summary())"""

'model = Sequential()\n\nmodel.add(Conv2D(6, (5,5),\n                 padding = \'same\',\n                 input_shape = input_shape))\nmodel.add(Activation("relu"))\nmodel.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))\n\nmodel.add(Conv2D(16, (5,5),\n                 padding = \'same\'))\nmodel.add(Activation("relu"))\nmodel.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))\n\nmodel.add(Conv2D(120, (5,5),\n                 padding = \'same\'))\nmodel.add(Activation("relu"))\nmodel.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))\n\nmodel.add(Flatten())\nmodel.add(Dense(120))\nmodel.add(Activation("relu"))\n\nmodel.add(Dense(84))\nmodel.add(Activation("relu"))\n\nmodel.add(Dense(num_classes))\nmodel.add(Activation("softmax"))\n\nmodel.compile(loss = \'categorical_crossentropy\',\n              optimizer = Adadelta(),\n              metrics = [\'accuracy\'])\n\nprint(model.summary())'

In [3]:
def build_model(hp):
    model = Sequential()
    for i in range(0, hp['num_layers']):
        model.add(Conv2D(hp['feature_maps'][i], (5,5),
                 padding = 'same',
                 input_shape = input_shape))
        model.add(Activation("relu"))
        model.add(AveragePooling2D(pool_size=(2,2), strides = (2,2)))
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Activation("relu"))

    model.add(Dense(84))
    model.add(Activation("relu"))

    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    return model

model_info = {'num_layers': 3,
              'feature_maps': [6, 16, 120]}

In [4]:
model = build_model(model_info)
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])
    
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 activation (Activation)     (None, 28, 28, 6)         0         
                                                                 
 average_pooling2d (Average  (None, 14, 14, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2416      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 16)        0         
                                                                 
 average_pooling2d_1 (Avera  (None, 7, 7, 16)          0         
 gePooling2D)                                           

In [7]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf

lr = 0.01

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr*tf.math.exp(-0.1)
    
schedule = LearningRateScheduler(scheduler, verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)
checkpoint = ModelCheckpoint("MNIST_Checkpoint.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
tensorboard = TensorBoard(log_dir="logs",
                          write_graph=True,
                          write_steps_per_second=True,
                          histogram_freq=5)

reg_callbacks = [schedule, earlystop, checkpoint, tensorboard]

In [1]:
%load_ext tensorboard

In [2]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
An attempt was made to access a socket in a way forbidden by its access permissions

In [12]:
tensorboard --logdir=logs/fit --port=8008

SyntaxError: cannot assign to expression (877742712.py, line 1)

In [4]:
pip show tensorboard

Name: tensorboard
Version: 2.9.0
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\indtal\anaconda3\envs\deeplearning\lib\site-packages
Requires: requests, tensorboard-data-server, protobuf, setuptools, werkzeug, numpy, wheel, absl-py, tensorboard-plugin-wit, google-auth, grpcio, markdown, google-auth-oauthlib
Required-by: tensorflow
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Training Parameters
batch_size = 128
epochs = 50

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=reg_callbacks,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save("mnist_LeNet.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
469/469 [==============================] - ETA: 0s - loss: 2.3019 - accuracy: 0.1156
Epoch 1: val_loss improved from inf to 2.30138, saving model to MNIST_Checkpoint.h5
469/469 [==============================] - 42s 90ms/step - loss: 2.3019 - accuracy: 0.1156 - val_loss: 2.3014 - val_accuracy: 0.1160 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
468/469 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 0.1158
Epoch 2: val_loss improved from 2.30138 to 2.30042, saving model to MNIST_Checkpoint.h5
469/469 [==============================] - 44s 93ms/step - loss: 2.3009 - accuracy: 0.1158 - val_loss: 2.3004 - val_accuracy: 0.1186 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
469/469 [==============================] - ETA: 0s - loss: 2.2999 - accuracy: 0.1234
Epoch 3:

KeyboardInterrupt: 